In [3]:
pip install yfinance statsmodels pandas numpy scipy

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 8.6 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15618 sha256=de89dc124a9e683af234033dbda69175a9a83a1805cd13449be3ea1b6b691de4
  Stored in directory: c:\users\akshath\appdata\local\pip\cache\wheels\1e\df\0f\e2bbb22d689b30c681feb5410ab64a2523437b34c8ecfc6476
Successfully built multitasking

   ---------------------------------------- 0/5 [peewee]
   ---------------------------------------- 0/5 [peewee]
   ---------------------------------------- 0/5 [peewee]
   ---------------- ----------------------- 2/5 [websockets]
   ---------------- ----------------------- 2/5 [websockets]
   ---------------- ----------------------- 2/5 [websockets]
   ---------------- ----------------------- 2/5 [websockets]
   -------

In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import mean_absolute_error

In [26]:
# 1. FETCH MARKET DATA
# Pulling the exact indices required
tickers = ["^GSPC", "^DJI", "^IXIC", "^RUT", "^TNX", "^VIX"]
# Download and select the 'Close' price specifically
raw_data = yf.download(tickers, start="1998-01-01", end="2020-04-01")['Close'].dropna()

[*********************100%***********************]  6 of 6 completed


In [27]:
# 2. DATA PREPARATION
# Log-transforming to stabilize variance (treat heteroscedasticity)
data = np.log(raw_data)
y = data["^GSPC"]  # S&P 500 Target
X = data[["^DJI", "^IXIC", "^RUT", "^TNX", "^VIX"]]  # Predictors

In [28]:
def run_stress_test(train_end, shock_start, shock_end):
    # Split into 'Normal' training and 'Shock' testing periods
    train_y, train_X = y[:train_end], X[:train_end]
    test_y, test_X = y[shock_start:shock_end], X[shock_start:shock_end]

    # 3. ARIMA(1,1,1) MODEL [cite: 573]
    # d=1 handles non-stationarity automatically via differencing
    model = sm.tsa.ARIMA(train_y, order=(1, 1, 1), exog=train_X)
    results = model.fit()

    # 4. FORECASTING & BACK-TRANSFORMATION
    forecast = results.get_forecast(steps=len(test_y), exog=test_X)
    predicted = np.exp(forecast.summary_frame()['mean'])
    actual = np.exp(test_y)

    # 5. STATISTICAL PROOF OF FAILURE [cite: 731-736]
    # T-test proves the model 'breaks' during systemic shocks
    t_stat, p_val = stats.ttest_rel(predicted, actual)
    mae = mean_absolute_error(actual, predicted) # Quantifying the "Miss"
    return p_val, mae

In [29]:
# --- EXECUTION ---
p_2008, mae_2008 = run_stress_test('2008-08-31', '2008-09-29', '2008-10-10')
p_2020, mae_2020 = run_stress_test('2020-01-31', '2020-02-24', '2020-03-06')

C:\Users\Akshath\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Akshath\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Akshath\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Akshath\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_ind

In [31]:
print(f"2008 Shock: P-Value = {p_2008:.10f} | MAE = {mae_2008:.2f} Index Points")
print(f"2020 Shock: P-Value = {p_2020:.10f} | MAE = {mae_2020:.2f} Index Points")

2008 Shock: P-Value = 0.0000008327 | MAE = 30.09 Index Points
2020 Shock: P-Value = 0.0000000610 | MAE = 24.47 Index Points
